In [23]:
import face_recognition
import dlib
import cv2
import imageio
import numpy as np
import os
from PIL import Image, ImageDraw

In [24]:
#The code below is adopted from https://blog.paperspace.com/facial-recognition-with-deep-learning/

def get_face_encodings(path_to_image):
   image=cv2.imread(path_to_image)
   detected_faces = face_detector(image, 1)
   shapes_faces = [shape_predictor(image, face) for face in detected_faces]
   return [np.array(face_recognition_model.compute_face_descriptor(image, face_pose, 1)) for face_pose in shapes_faces]

def compare_face_encodings(known_faces, face):
    a=np.linalg.norm(known_faces - face, axis=1)   
    return [a,(a <= TOLERANCE)]

def find_match(known_faces, names, face):
   [a,matches] = compare_face_encodings(known_faces, face)
   print(a,names)
   count = 0
   for match in matches:
       if match:
           return names[count]
       count += 1
   return print('Not Found')

In [25]:
# Import Face Detector from dlib, to find faces in the images
face_detector = dlib.get_frontal_face_detector()
# Import Shape Predictor, that detects facial landmark points
shape_predictor = dlib.shape_predictor('/Users/alexfion/sciebo/software_neuro/face_reco/shape_predictor_68_face_landmarks.dat')
# Import the Face Recognition Model, which delivers the face encodings 
face_recognition_model = dlib.face_recognition_model_v1('/Users/alexfion/sciebo/software_neuro/face_reco/dlib_face_recognition_resnet_model_v1.dat')
# Tolerance describes the highest distance that faces can have, to still be considered a match. 
# Lower tolerance is more restrictive, and 0.6 idelivers typically the best performance.
TOLERANCE = 0.7

In [26]:
#import image filenames from the "real" pictures
image_filenames = filter(lambda x: x.endswith('.png'), os.listdir('/Users/alexfion/sciebo/software_neuro/face_reco/known/'))
image_filenames = sorted(image_filenames)
paths_to_images = ['/Users/alexfion/sciebo/software_neuro/face_reco/known/' + x for x in image_filenames]
face_encodings = []
for path_to_image in paths_to_images:
   face_encodings_in_image = get_face_encodings(path_to_image)
   if len(face_encodings_in_image) != 1:
       print("Please change image: " + path_to_image + " - it has " + str(len(face_encodings_in_image)) + " faces; it can only have one")
       exit()
   face_encodings.append(get_face_encodings(path_to_image)[0])

#import image filenames from the "MRI" pictures
test_filenames = filter(lambda x: x.endswith('.png'), os.listdir('/Users/alexfion/sciebo/software_neuro/face_reco/test/'))
paths_to_test_images = ['/Users/alexfion/sciebo/software_neuro/face_reco/test/' + x for x in test_filenames]
names = [x[:-4] for x in image_filenames]

#The output returns estimated distances between faces for every MRI reconstruction, as well as names of "known" photographs,
#against which these reconstractions were tested. Also, the best fit is returned
for path_to_image in paths_to_test_images:
   face_encodings_in_image = get_face_encodings(path_to_image)
   if len(face_encodings_in_image) != 1:
       print("Please change image: " + path_to_image + " - it has " + str(len(face_encodings_in_image)) + " faces; it can only have one")
       exit()
   match = find_match(face_encodings, names, face_encodings_in_image[0])
   print(path_to_image, match)

[0.74466014 0.64071178 0.69777809 0.67272186 0.69934906 0.73731642] ['34049718_2215508268459277_5295707238747340800_n', 'Photo', 'Photo_2', 'Untitled', 'cv_photo', 'picture1']
/Users/alexfion/sciebo/software_neuro/face_reco/test/reco_alex_2021_sharp_not_bright.png Photo
[0.7558248  0.70421275 0.69146095 0.69822437 0.73469024 0.80501742] ['34049718_2215508268459277_5295707238747340800_n', 'Photo', 'Photo_2', 'Untitled', 'cv_photo', 'picture1']
/Users/alexfion/sciebo/software_neuro/face_reco/test/91693755_nose1_4_0.18_1_0.3.png Photo_2
[0.73557737 0.64785699 0.68952439 0.66973352 0.71700798 0.80457494] ['34049718_2215508268459277_5295707238747340800_n', 'Photo', 'Photo_2', 'Untitled', 'cv_photo', 'picture1']
/Users/alexfion/sciebo/software_neuro/face_reco/test/reco_alex.png Photo
[0.70652957 0.58726632 0.65023255 0.6347072  0.61542897 0.65791727] ['34049718_2215508268459277_5295707238747340800_n', 'Photo', 'Photo_2', 'Untitled', 'cv_photo', 'picture1']
/Users/alexfion/sciebo/software_neu

Let's demonstrate the overlap between two faces with the best overlap. This is the case for the MRI reconstruction "reco_MRI.png" and the photograph "Photo.png". 

In [28]:
known_image = face_recognition.load_image_file("/Users/alexfion/sciebo/software_neuro/face_reco/known/Photo.png")
unknown_image = face_recognition.load_image_file("/Users/alexfion/sciebo/software_neuro/face_reco/test/reco_MRI.png")

known_encoding = face_recognition.face_encodings(known_image)[0]
unknown_encoding = face_recognition.face_encodings(unknown_image)[0]
#under threshold of 0.6, do the faces overlap? 
results = face_recognition.compare_faces([known_encoding], unknown_encoding,0.6)
print(results)

[True]


The face distance in this case is:

In [29]:
face_distances = face_recognition.face_distance(face_recognition.face_encodings(known_image), unknown_encoding)
face_distances

array([0.59155943])

Let's look for faces in the pictures:

In [33]:
face_locations = face_recognition.face_locations(unknown_image, number_of_times_to_upsample=0, model="cnn")
for face_location in face_locations:
    top, right, bottom, left = face_location
    face_image = unknown_image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    pil_image.show()

In [32]:
face_locations = face_recognition.face_locations(known_image, number_of_times_to_upsample=0, model="cnn")
for face_location in face_locations:
    top, right, bottom, left = face_location
    face_image = known_image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    pil_image.show()

Let's also draw detect faces in the images: 

In [34]:
image = face_recognition.load_image_file("/Users/alexfion/sciebo/software_neuro/face_reco/known/Photo.png")
face_landmarks_list = face_recognition.face_landmarks(image)  
face_landmarks = face_landmarks_list[0]  
im = Image.open("/Users/alexfion/sciebo/software_neuro/face_reco/known/Photo.png")
draw = ImageDraw.Draw(im)

# Draw all the features for the first face
draw.line(face_landmarks['chin'])
draw.line(face_landmarks['left_eyebrow'])
draw.line(face_landmarks['right_eyebrow'])
draw.line(face_landmarks['nose_bridge'])
draw.line(face_landmarks['nose_tip'])
draw.line(face_landmarks['left_eye'])
draw.line(face_landmarks['right_eye'])
draw.line(face_landmarks['top_lip'])
draw.line(face_landmarks['bottom_lip'])
im.show()


In [35]:
image = face_recognition.load_image_file("/Users/alexfion/sciebo/software_neuro/face_reco/test/reco_MRI.png")
face_landmarks_list = face_recognition.face_landmarks(image)  
face_landmarks = face_landmarks_list[0]  
im = Image.open("/Users/alexfion/sciebo/software_neuro/face_reco/test/reco_MRI.png")
draw = ImageDraw.Draw(im)
# Draw all the features for the second face
draw.line(face_landmarks['chin'])
draw.line(face_landmarks['left_eyebrow'])
draw.line(face_landmarks['right_eyebrow'])
draw.line(face_landmarks['nose_bridge'])
draw.line(face_landmarks['nose_tip'])
draw.line(face_landmarks['left_eye'])
draw.line(face_landmarks['right_eye'])
draw.line(face_landmarks['top_lip'])
draw.line(face_landmarks['bottom_lip'])
im.show()


This way we see, that a significant overlap exists between reconstructed MRI faces and actual photos. 